In [0]:
import numpy as np
import torch

In [0]:
# Loading Inputs
inputs = np.array([
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]
],dtype='float32')

In [0]:
# Loading Targets
targets = np.array([
                    [56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]
],dtype='float32')

In [4]:
# Converting from numpy to tensor type
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print('Inputs: \n',inputs)
print('Targets: \n',targets)

Inputs: 
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
Targets: 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [5]:
# Weights and Biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print('w: \n',w)
print('b: \n',b)

w: 
 tensor([[-1.3104, -0.0317,  0.8074],
        [ 0.7401, -1.4553, -0.3729]], requires_grad=True)
b: 
 tensor([-0.7928,  2.0275], requires_grad=True)


In [6]:
# Printing all the values
print('Inputs: \n',inputs,
      '\nTargets: \n',targets,
      '\nw: \n',w,
      '\nb: \n',b)

Inputs: 
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]) 
Targets: 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]) 
w: 
 tensor([[-1.3104, -0.0317,  0.8074],
        [ 0.7401, -1.4553, -0.3729]], requires_grad=True) 
b: 
 tensor([-0.7928,  2.0275], requires_grad=True)


In [0]:
# Defining the model
def model(x):
    return x@w.t() + b

In [8]:
# Generating the predictions
preds = model(inputs)
print('Predictions: \n', preds)

Predictions: 
 tensor([[ -63.8524,  -57.4898],
        [ -71.1487,  -82.5619],
        [ -72.2077, -150.2290],
        [-105.9372,    1.1377],
        [ -37.7299, -112.7237]], grad_fn=<AddBackward0>)


In [9]:
#Comparing with targets
print("Difference between the TARGETS and PREDICTIONS: \nTARGETS-PREDICTOINS: \n", targets-preds)

Difference between the TARGETS and PREDICTIONS: 
TARGETS-PREDICTOINS: 
 tensor([[119.8524, 127.4898],
        [152.1487, 183.5619],
        [191.2077, 283.2289],
        [127.9372,  35.8624],
        [140.7299, 231.7237]], grad_fn=<SubBackward0>)


So we Got a huge difference with the predictions and the targets
**But** we can't compare in this manner beacuse we don't know which array has the highest or tghe lowest elements or are they containing negative element too. so we can calculate the difference between the predictions and the targets with the help of **LOSS FUNCTION** i.e., **MSE** (Mean Square Error)Loss

In [0]:
# Lets Calculate the loss with the LOSS functions
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff*diff)/diff.numel()

In [11]:
# Lets compute the loss
loss = mse(preds, targets)
print('Loss: \n', loss)

Loss: 
 tensor(29539.6250, grad_fn=<DivBackward0>)


# **------------Gradient Descent------------**start

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`
**Compute gradients**

In [0]:
# Compute gradients
loss.backward()

The calculated gradients are stored in the `.grad` preoperty of the respective tensors

In [13]:
# Gradients for weights
print('W: \n', w)
print('Calculated gradient of W: \n', w.grad)

W: 
 tensor([[-1.3104, -0.0317,  0.8074],
        [ 0.7401, -1.4553, -0.3729]], requires_grad=True)
Calculated gradient of W: 
 tensor([[-12397.9570, -13210.4795,  -8113.1978],
        [-14059.7402, -17287.0996, -10240.9736]])


The `loss` is a quadratic function of our `weights` and `biases`, and **our objective is to find the set of weights where the loss is the lowest**. 

If a gradient element is **positive**:

- **increasing** the element's value slightly will **increase** the loss.

- **decreasing** the element's value slightly will **decrease** the loss

![Graph1](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:

**increasing** the element's value slightly will **decrease** the loss.

**decreasing** the element's value slightly will **increase** the loss.

![Graph2](https://i.imgur.com/w3Wii7C.png)

Before we proceed, we reset the gradients to zero by calling .zero_() method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.


In [35]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# **In any Linear regression model our main aim is to adjust the weights and the biasses** Using **gradient descent to reduce the loss and increase our accuracy percentage**

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [14]:
# 1. Generate predictions
preds = model(inputs)
print('Prediction: \n',preds)

Prediction: 
 tensor([[ -63.8524,  -57.4898],
        [ -71.1487,  -82.5619],
        [ -72.2077, -150.2290],
        [-105.9372,    1.1377],
        [ -37.7299, -112.7237]], grad_fn=<AddBackward0>)


In [15]:
# 2. Calculate the loss
loss = mse(preds, targets)
print('Loss: \n', loss)

Loss: 
 tensor(29539.6250, grad_fn=<DivBackward0>)


In [16]:
# 3. Compute loss gradients w.r.t. weights and biases\
loss.backward()  #This computes the gradient of whichever variable the `requires_grad` is set to `True`
print('W Gradient: ',w.grad)
print('b Gradient: ',b.grad)

W Gradient:  tensor([[-24795.9141, -26420.9590, -16226.3955],
        [-28119.4805, -34574.1992, -20481.9473]])
b Gradient:  tensor([-292.7504, -344.7467])


In [0]:
# 4. Adjust the weights by subtracting a small quantity proportional to the gradient
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

- We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

- We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm. 

- After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

In [19]:
print(w)
print(b)

tensor([[-0.8144,  0.4968,  1.1319],
        [ 1.3025, -0.7638,  0.0367]], requires_grad=True)
tensor([-0.7898,  2.0310], requires_grad=True)


**------------Gradient Descent---------------**End

**Now with the new weights and biases, the model should have lower loss**

In [22]:
# Calculating the loss again to see how much we are able to reduce the loss
preds = model(inputs) # we need to initialize this line againg becuase inside the model function b amd w are persent and then only this will take efffect with our new w, b
loss = mse(preds, targets)
print(loss)

tensor(3951.8406, grad_fn=<DivBackward0>)


# Train for Multiple Epochs

In [0]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [25]:
# Lets calculate the loss after 100 epochs
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(489.4872, grad_fn=<DivBackward0>)


In [26]:
# Lets compare the preds and the targets again
print('Prediction: \n', preds)
print('Target: \n', targets)

Prediction: 
 tensor([[ 56.0796,  80.5704],
        [ 85.7269, 104.2248],
        [112.4302, 108.3839],
        [ 15.9675,  94.3433],
        [110.8728,  92.2672]], grad_fn=<AddBackward0>)
Target: 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# **Linear Regression using PyTorch built-ins** start


In [0]:
import torch.nn as nn

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Dataset and DataLoader
We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as `tuples`, and provides standard APIs for working with many different types of datasets in PyTorch.

In [0]:
from torch.utils.data import TensorDataset

In [30]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [0]:
from torch.utils.data import DataLoader

In [0]:
# Defining data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [33]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]])
tensor([[ 56.,  70.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 22.,  37.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

# nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [34]:
# Defining Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2614, -0.3836, -0.5516],
        [ 0.5116,  0.1868, -0.1979]], requires_grad=True)
Parameter containing:
tensor([0.4856, 0.3416], requires_grad=True)


In [35]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2614, -0.3836, -0.5516],
         [ 0.5116,  0.1868, -0.1979]], requires_grad=True),
 Parameter containing:
 tensor([0.4856, 0.3416], requires_grad=True)]

In [36]:
# Lets generate Predictions
preds = model(inputs)
preds

tensor([[-29.8472,  41.6933],
        [-44.7794,  50.6682],
        [-60.1606,  58.4001],
        [ -9.7505,  53.2366],
        [-56.9088,  39.7184],
        [-29.8472,  41.6933],
        [-44.7794,  50.6682],
        [-60.1606,  58.4001],
        [ -9.7505,  53.2366],
        [-56.9088,  39.7184],
        [-29.8472,  41.6933],
        [-44.7794,  50.6682],
        [-60.1606,  58.4001],
        [ -9.7505,  53.2366],
        [-56.9088,  39.7184]], grad_fn=<AddmmBackward>)

# Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [0]:
# import nn.functional
import torch.nn.functional as F

In [0]:
# Define loss function
loss_fn = F.mse_loss

In [39]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9731.6504, grad_fn=<MseLossBackward>)


# Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in batches (often with random shuffling) instead of as a single group.

In [0]:
# Defining Optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

# Train the model
We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function fit which trains the model for a given number of epochs.

In [0]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [43]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 616.8636
Epoch [20/100], Loss: 910.7531
Epoch [30/100], Loss: 382.6708
Epoch [40/100], Loss: 376.0389
Epoch [50/100], Loss: 273.8224
Epoch [60/100], Loss: 9.3404
Epoch [70/100], Loss: 139.7085
Epoch [80/100], Loss: 94.6256
Epoch [90/100], Loss: 77.4458
Epoch [100/100], Loss: 67.7181


In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.2414,  72.0684],
        [ 79.0050,  96.4597],
        [121.9039, 138.7687],
        [ 32.7958,  47.1114],
        [ 89.6632, 106.0563],
        [ 59.2414,  72.0684],
        [ 79.0050,  96.4597],
        [121.9039, 138.7687],
        [ 32.7958,  47.1114],
        [ 89.6632, 106.0563],
        [ 59.2414,  72.0684],
        [ 79.0050,  96.4597],
        [121.9039, 138.7687],
        [ 32.7958,  47.1114],
        [ 89.6632, 106.0563]], grad_fn=<AddmmBackward>)

In [45]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# **Linear Regression using PyTorch built-ins** End



For an animated visualization of how linear regression works, [see this post.](https://jovian.ml/outlink?url=https%3A%2F%2Fhackernoon.com%2Fvisualizing-linear-regression-with-pytorch-9261f49edb09)

For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://jovian.ml/outlink?url=https%3A%2F%2Fgithub.com%2FCleo-Stanford-CS%2FCS229_Notes%2Fblob%2Fmaster%2Flectures%2Fcs229-notes1.pdf) from CS229 at Stanford University.

To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://jovian.ml/outlink?url=https%3A%2F%2Fwww.kaggle.com%2Fc%2Fboston-housing) competition on Kaggle, a website that hosts data science competitions.